# Measurement with custom parameters

In this demo, I'll show you how to create a measurement without a Station or Instruments. You can create any custom **Parameter** and run a measurement and save the result in a **DataSet**.

In [ ]:
import qcodes as qc
import numpy as np
from qcodes.dataset.plotting import plot_dataset
from time import time, sleep
import tqdm

In [ ]:
import qsharp
qsharp.packages.add("Microsoft.Quantum.Chemistry")
qsharp.reload()

## Import Q# program

Using the Python interoperability, import the Q# program into your namespace.

In [ ]:
from Microsoft.Quantum.Samples.Chemistry.SimpleVQE import GetEnergyHydrogenVQE

## Initialize database

Initialize the database and load a new experiment.

In [ ]:
qc.initialise_or_create_database_at("./experiments.db")
exp = qc.load_or_create_experiment(experiment_name='VQE',
                                sample_name="Q# simulator")

In [ ]:
qsharp.__version__

## Run the experiment

You can now go ahead and run the experiment! Use `Measurement.register_custom_parameter` to create custom parameters that don't depend on an **Instrument**.

In [ ]:
ystart, ystop, ystep = -2., 2., .5
zstart, zstop, zstep = -2., 2., .5
x = 0.0
ydata = np.arange(ystart, ystop+ystep, ystep)
zdata = np.arange(zstart, zstop+zstep, zstep)
n_samples = 10e3

# Create the initial parameters (noting that our single qubit variational form has 3 parameters)
meas = qc.Measurement(exp=exp, station=None, name='VQE measurement')
meas.register_custom_parameter("theta1")
meas.register_custom_parameter("theta2")
meas.register_custom_parameter("theta3")
meas.register_custom_parameter("energy", setpoints=("theta2", "theta3"))
meas.write_period = 2 

with meas.run() as datasaver:
    values = [(x, y, z) for y in ydata for z in zdata]
    for theta in tqdm.tqdm(values):
        theta1, theta2, theta3 = theta
        energy = GetEnergyHydrogenVQE(theta1=theta1, theta2=theta2, theta3=theta3, nSamples=n_samples)
        datasaver.add_result(
            ("theta1", theta1),
            ("theta2", theta2),
            ("theta3", theta3),
            ("energy", energy)        
        )
        sleep(0.2)
    dataset = datasaver.dataset  # convenient to have for plotting

## Plot the data

Use plot_dataset to plot the data.

In [ ]:
plot_dataset(dataset)

## Create a VQE program

Create a VQE program that uses scipy for the optimization loop.

In [ ]:
from scipy.optimize import minimize

def VQE(initial_var_params, n_samples, **kwargs):
    """ Run VQE Optimization to find the optimal energy and the associated variational parameters """
    
    # Create the initial parameters (noting that our single qubit variational form has 3 parameters)
    meas = qc.Measurement(exp=exp, station=None, name='VQE measurement')
    meas.register_custom_parameter("time")
    meas.register_custom_parameter("theta1")
    meas.register_custom_parameter("theta2")
    meas.register_custom_parameter("theta3")
    meas.register_custom_parameter("energy", setpoints=("time",))
    meas.write_period = 2 

    time_start = time()
    
    with meas.run() as datasaver:
        def get_data(theta, n_samples):
            theta1, theta2, theta3 = theta
            energy = GetEnergyHydrogenVQE(theta1=theta1, theta2=theta2, theta3=theta3, nSamples=n_samples)
            datasaver.add_result(
                ("time", time() - time_start),
                ("theta1", theta1),
                ("theta2", theta2),
                ("theta3", theta3),
                ("energy", energy)        
            )
            return energy

        opt_result = minimize(get_data,
                              initial_var_params,
                              args=(n_samples),
                              **kwargs)
        dataset = datasaver.dataset  # convenient to have for plotting

    return opt_result, dataset

## Run VQE program

Run the program. Here we use Nelder-Mead for the optimization.

In [ ]:
opt_result, dataset = VQE(initial_var_params=(0.0, -.5, 0.7), n_samples=10000, 
                          method="nelder-mead",
                            options={
                              'maxiter': 200,
                              'maxfev': None,
                              'disp': False,
                              'return_all': False,
                              'initial_simplex': None,
                              'xatol': 0.01,
                              'fatol': 0.01,
                              'adaptive': False
                            })

## Plot data

Again, we can go ahead and plot the data.

In [ ]:
plot_dataset(dataset)

## Run VQE program

Run the program. Here we use COBYLA for the optimization.

In [ ]:
opt_result, dataset = VQE(
    initial_var_params=(0.0, -.5, 0.7),
    n_samples=10000,
    method="COBYLA",
    tol=0.000001,
    options={'disp': True, 'maxiter': 200,'rhobeg' : 0.01}
)

In [ ]:
plot_dataset(dataset)

## View data in experimental widget

In [ ]:
from qcodes.interactive_widget import experiments_widget
experiments_widget()